# Process NCRMP Puerto Rico Disease Data

This manuscript dataset has a total number of corals for each species, and counts of disease presence. The scipt coneverts raw data from the manuscript into Tidy format(e.g. filling in genus name in each row). Using the diseased and healthy counts to infer the total disease percentages.  

## Import data as a pandas DataFrame

In [1]:
import pandas as pd
Puerto_Rico_data= pd.read_csv("../Raw_Data/NCRMP_Puerto_Rico_raw.csv")
print(Puerto_Rico_data.columns)

Index(['time', 'latitude', 'longitude', 'REGION', 'REGION_DESCRIPTION',
       'PRIMARY_SAMPLE_UNIT', 'STATION_NR', 'YEAR', 'MONTH', 'DAY', 'Date_UTC',
       'HABITAT_CD', 'HABITAT_TYPE', 'STRAT', 'STRAT_Description',
       'RUGOSITY_CD', 'WTD_RUG', 'MEAN_RUG', 'MAPGRID_NR', 'SUB_REGION_NAME',
       'SUB_REGION_NAME_DESCRIPTION', 'SUB_REGION_NR', 'ZONE_NAME', 'ZONE_NR',
       'MPA_NAME', 'MPA_NR', 'ADMIN', 'Administration_Description', 'PROT',
       'DEPTH_STRAT', 'DEPTH_STRAT_DESCRIPTION', 'MIN_DEPTH', 'MAX_DEPTH',
       'METERS_COMPLETED', 'SPECIES_CD', 'SPECIES_NAME', 'N', 'JUV',
       'MAX_DIAMETER', 'PERP_DIAMETER', 'HEIGHT', 'OLD_MORT', 'RECENT_MORT',
       'BLEACH_CONDITION', 'DISEASE', 'accession_url'],
      dtype='object')


/var/folders/17/9kvd528n47j282gcln4v5zvc0000gp/T/ipykernel_1636/2164083636.py:2: DtypeWarning: Columns (1,2,17,31,32,33,38,39,40,41,42) have mixed types. Specify dtype option on import or set low_memory=False.
  Puerto_Rico_data= pd.read_csv("../Raw_Data/NCRMP_Puerto_Rico_raw.csv")


## Isolate desired columns

In [2]:
Puerto_Rico_data = Puerto_Rico_data[["SPECIES_CD","SPECIES_NAME","DISEASE"]]
Puerto_Rico_data = Puerto_Rico_data.dropna(how='all').reset_index(drop=True)
Puerto_Rico_data

,SPECIES_CD,SPECIES_NAME,DISEASE
0,ACR CERV,Acropora cervicornis,NaN
1,ACR CERV,Acropora cervicornis,NaN
2,ACR CERV,Acropora cervicornis,NaN
3,ACR CERV,Acropora cervicornis,NaN
4,ACR CERV,Acropora cervicornis,NaN
...,...,...,...
41450,ACR PALM,Acropora palmata,NaN
41451,ACR CERV,Acropora cervicornis,NaN
41452,ACR CERV,Acropora cervicornis,NaN
41453,ACR CERV,Acropora cervicornis,NaN


## Selecting Disease column and setting empty cells to healthy

In [3]:
Puerto_Rico_data['DISEASE'][Puerto_Rico_data['DISEASE'].isnull()] = "Unknown"
Puerto_Rico_data

,SPECIES_CD,SPECIES_NAME,DISEASE
0,ACR CERV,Acropora cervicornis,Unknown
1,ACR CERV,Acropora cervicornis,Unknown
2,ACR CERV,Acropora cervicornis,Unknown
3,ACR CERV,Acropora cervicornis,Unknown
4,ACR CERV,Acropora cervicornis,Unknown
...,...,...,...
41450,ACR PALM,Acropora palmata,Unknown
41451,ACR CERV,Acropora cervicornis,Unknown
41452,ACR CERV,Acropora cervicornis,Unknown
41453,ACR CERV,Acropora cervicornis,Unknown


## Count corals by genus and Disease column

In [4]:
Grouped_Puerto_Rico_data = Puerto_Rico_data.groupby(['SPECIES_NAME','DISEASE']).agg(Disease_count = ("DISEASE", "count")).reset_index()

Grouped_Puerto_Rico_data

,SPECIES_NAME,DISEASE,Disease_count
0,Acropora cervicornis,A,76
1,Acropora cervicornis,P,7
2,Acropora cervicornis,Unknown,572
3,Acropora palmata,A,27
4,Acropora palmata,P,2
...,...,...,...
159,Tubastraea coccinea,A,4
160,Tubastraea coccinea,Unknown,269
161,Undaria spp,A,23
162,Undaria spp,P,2


## Create Disease table 

We need to organise the data so we are able to calculate total disease percent and include a genus column. 

In [5]:
from numpy import isfinite
from math import isnan
disease_df = Puerto_Rico_data
unique_diseases = set(disease_df["DISEASE"].unique())

df = Puerto_Rico_data
unique_species = set(df["SPECIES_NAME"].unique())
unique_species = list(unique_species)
print(unique_species)
print(unique_diseases)

disease_table = pd.DataFrame(0, index=unique_species,columns=unique_diseases)
for species_name in unique_species:
    for disease in unique_diseases: 
        disease_subtable = df.loc[(df['SPECIES_NAME'] == species_name) & (df['DISEASE'] == disease)]
        empty = bool(disease_subtable.empty)
        if not empty:
            count = disease_subtable['DISEASE'].count()
        
            disease_table.loc[species_name, disease] = count
disease_table.sort_index()

disease_table = disease_table.rename(columns ={'A':'healthy_count', 'P':'total_diseased_count', 'Unknown':'unknown_count'})
disease_table

disease_table["total_diseased_percent"] = disease_table["total_diseased_count"]/(disease_table["healthy_count"]+ disease_table["total_diseased_count"])

disease_table = disease_table.drop(columns=['F','S'])
disease_table = disease_table.rename_axis('species')


disease_table['genus'] = disease_table.index.str.split().str[0] #add to other similar sets 
#move total disease column to right hand side of genus 
disease_table = disease_table[['genus','total_diseased_percent', 'healthy_count','total_diseased_count','unknown_count']]

disease_table

['Agaricia fragilis', 'Dichocoenia stokesii', 'Other coral', 'Mycetophyllia aliciae', 'Mycetophyllia lamarckiana', 'Madracis senaria', 'Orbicella annularis', 'Agaricia agaricites', 'Pseudodiploria strigosa', 'Siderastrea radians', 'Favia fragum', 'Solenastrea bournoni', 'Mycetophyllia danaana', 'Isophyllastrea rigida', 'Meandrina spp', 'Colpophyllia natans', 'Madracis pharensis', 'Madracis carmabi', 'Mussa angulosa', 'Tubastraea coccinea', 'Siderastrea siderea', 'Meandrina danae', 'Orbicella annularis species complex', 'Siderastrea spp', 'Scolymia cubensis', 'Meandrina meandrites', 'Meandrina jacksoni', 'Scolymia spp', 'Mycetophyllia spp', 'Madracis decactis', 'Undaria spp', 'Madracis spp', 'Orbicella spp', 'Acropora prolifera', 'Isophyllia sinuosa', 'Montastraea cavernosa', 'Porites porites', 'Oculina diffusa', 'Acropora palmata', 'Manicina areolata', 'Madracis formosa', 'Diploria labyrinthiformis', 'Pseudodiploria clivosa', 'Orbicella franksi', 'Acropora cervicornis', 'Porites divari

,genus,total_diseased_percent,healthy_count,total_diseased_count,unknown_count
species,,,,,
Agaricia fragilis,Agaricia,0.013699,72,1,566
Dichocoenia stokesii,Dichocoenia,0.022099,177,4,479
Other coral,Other,0.000000,2,0,332
Mycetophyllia aliciae,Mycetophyllia,0.000000,11,0,434
Mycetophyllia lamarckiana,Mycetophyllia,0.000000,2,0,268
...,...,...,...,...,...
Mycetophyllia ferox,Mycetophyllia,0.000000,8,0,597
Porites astreoides,Porites,0.010497,4619,49,121
Agaricia lamarcki,Agaricia,0.007937,125,1,551


In [6]:
disease_table.to_csv("../Processed_Data/NCRMP_Puerto_Rico_corals.csv",index_label="species")

## Summarize disease at genus level

In [8]:
final_column_names = ['genus', 'healthy_count', 'total_diseased_count','unknown_count']

genus_table = disease_table.loc[:,final_column_names]
genus_table = genus_table.groupby('genus')['healthy_count','total_diseased_count','unknown_count'].sum().reset_index()
genus_table['total_diseased_percent'] = genus_table['total_diseased_count']/(genus_table['healthy_count'] + genus_table['total_diseased_count'])
genus_table = genus_table.dropna()
genus_table.to_csv("../Processed_Data/per_genus_disease_data/NCRMP_Puerto_Rico_corals.csv",index_label="genus")

genus_table

/var/folders/17/9kvd528n47j282gcln4v5zvc0000gp/T/ipykernel_1636/3025703925.py:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  genus_table = genus_table.groupby('genus')['healthy_count','total_diseased_count','unknown_count'].sum().reset_index()


,genus,healthy_count,total_diseased_count,unknown_count,total_diseased_percent
0,Acropora,106,9,1273,0.078261
1,Agaricia,2227,15,3402,0.006690
2,Colpophyllia,93,2,537,0.021053
3,Dendrogyra,20,1,585,0.047619
4,Dichocoenia,177,4,479,0.022099
5,Diploria,162,3,508,0.018182
6,Eusmilia,48,0,562,0.000000
7,Favia,18,0,591,0.000000
8,Helioceris,110,0,542,0.000000
9,Isophyllastrea,6,0,426,0.000000
